# Experiment on Absorbing Boundary Conditions and Perfectly Matched Layers

---
## 1 - Introduction

This document presents the results of our simulations. The source code can be found in the sub-files

The organization of this experimental paper follows that of the review of the article [*A Friendly Review of Absorbing Boundary Conditions and Perfectly Matched Layers for Classical and Relativistic Quantum Waves Equations*](https://www.tandfonline.com/doi/abs/10.1080/00268976.2017.1290834) by X. Antoinea, E. Lorinb and Q. Tanga.

The code was written by Linnea Hallin, Maxime Renard, Eloi Navet & Nicolas Roblet under the direction of Ms Brigitte Bidegaray-Fesquet and Mr Clément Jourdana.

This file serves as an exhibition of the code produced. Its structure follows that of the project report.

*Note:* The python version used for development is the 3.10.12.

## 2 - One-dimensional Wave equation
We simulate the homogoneous diffusion equation
\begin{equation}
    \left \{\begin{array}{ll}
        \partial_{tt}\psi - c^2 \partial_{xx}\psi = 0 \hspace{2.25cm} \text{in} \quad\R\times(0,+\infty),\\
        \psi(\cdot,0) = \psi_0,~~ \partial_t\psi(\cdot,0) = \psi_{t,0}\quad~~~\text{on}\quad \R.
    \end{array}\right.
\end{equation}

Let's then define simulation variable.

In [ ]:
import numpy as np

# Importation of the function
from wave_eq.wave_simul_implementation import main_experience as wave_experience

# Time mesh
period_to_emulate = 0.5
T_nb_point = 1000

# Space mesh
space_interval = (-1, 2)
X_nb_point = 500

# Celerity
c = 6

# Point source
Q = np.zeros(X_nb_point)

The proposed initial condition $\psi_0$ is a **double gaussian** as following and $\psi_{t,0}$ **null**.  

In [3]:
import matplotlib.pyplot as plt

# Initial condition (a default function is implemented)
def double_gaussian(x):
    psi_0_x = np.exp(-((x - (space_interval[0] + 0.2 * (space_interval[1] - space_interval[0])))** 2)/ (0.1**2))
    psi_0_x += np.exp(-((x - (space_interval[0] + 0.6 * (space_interval[1] - space_interval[0])))** 2)/ (0.1**2))
    return psi_0_x

psi_0 = double_gaussian
dtpsi_0 = lambda x: 0

interval_inf, interval_sup = space_interval
x = np.linspace(interval_inf, interval_sup, X_nb_point)

plt.plot(x, psi_0(x))
plt.title(
    r"Graph of $\psi_0$"
)
plt.xlabel("x")
plt.show()

### 2.2) Classic boundary conditions
#### 2.2.1) Dirichlet boundary condition

In [4]:
# Value at boundaries
bc_right_left_value = (0, 0)

# Launch code
wave_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    bctype="dirichlet",
    bc_right_left_value=bc_right_left_value,
    Q=Q,
);

#### 2.2.2) Periodic boundary condition

In [5]:
# Launch code
wave_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    bctype="periodic",
    Q=Q,
);

#### 2.2.3) Neumann boundary condition

In [6]:
# Value at boundaries
bc_right_left_value = (0, 0)

# Launch code
wave_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    bctype="neumann",
    bc_right_left_value=bc_right_left_value,
    Q=Q,
);

#### 2.3) Adding a source term

In [7]:
tmp_x = np.linspace(space_interval[0], space_interval[-1], X_nb_point)
Q = lambda x: np.exp(-100.0 * x**2)
psi_test_Q = np.zeros(X_nb_point)

# Value at boundaries
bc_right_left_value = (0, 0)

# Launch code
wave_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_test_Q,
    bctype="neumann",
    bc_right_left_value=bc_right_left_value,
    Q=Q,
);

### 2.4) Absorbing boundary condition

In [8]:
# Launch code
wave_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    bctype="transparent",
    # Q = Q
);

### 2.5) Study of the complete initial value problem
Let now consder a initial condition $\psi_{t,0}$ **non null**.  
To do this, let's consider, as explained in the study
$$
\forall x \in \R, \qquad \psi_{t,0}(x) = -c \psi_0'(x).
$$

In [9]:
def double_gaussian_deriv(x, c = c):
    def gaussian_term(x, center, width):
        return -2 * (x - center) * np.exp(-((x - center) ** 2) / (width ** 2)) / (width ** 2)

    psi_0_x = gaussian_term(x, space_interval[0] + 0.2 * (space_interval[1] - space_interval[0]), 0.1)
    psi_0_x += gaussian_term(x, space_interval[0] + 0.6 * (space_interval[1] - space_interval[0]), 0.1)

    return -c * psi_0_x

dtpsi_0 = lambda x : double_gaussian_deriv(x, c)

plt.plot(x, dtpsi_0(x))
plt.title(r"Graph of $\psi_0'$")
plt.show()

In [10]:
wave_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    dtpsi=dtpsi_0,
    bctype="transparent",
);

### 2.6) Study of the error
#### 2.6.1) Exact error

In [11]:
from wave_eq.wave_study_error import main_experience_error as wave_error_study

res_exact, res_simul, abs_error = wave_error_study(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    # dtpsi=dtpsi_0,
    bctype="transparent",
);

In [12]:
%%capture
%matplotlib inline
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"

nb_frames = 30
dt = period_to_emulate/T_nb_point

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
def animate(i):
    ind = int(i * T_nb_point / nb_frames)
    ax[0].clear()
    ax[1].clear()
    ax[0].plot(x, np.abs(res_exact[ind]), label = "exact")
    ax[0].plot(x, np.abs(res_simul[ind]), label = "simul")
    ax[1].plot(x, abs_error[ind], label = "error")
    ax[0].set_title(r'Evolution of wave equation')
    ax[0].set_xlabel('x')
    ax[0].set_ylim(-1.1, 1.1)
    ax[0].set_ylabel(r'$|\psi(t,\cdot)|$')
    ax[1].set_title('Error')
    ax[1].set_xlabel('x')
    ax[1].set_ylabel('Time')
    ax[0].legend()
    ax[1].legend()
    fig.suptitle(f"State at t={(ind*dt):.6g}", fontsize=16)

ani = animation.FuncAnimation(fig, animate, frames=nb_frames)
ani

#### 2.6.2) Isolating reflections

The notation $\psi$ is used to denote the wave function computed as before, and $\phi$ to one computed on a larger domain. Then, to isolate reflections, the function $\psi - \phi|_\Omega$ is plotted.

In [14]:
from wave_eq.wave_study_error import isolate_reflections

isolate_reflections(
    period_to_emulate=1.0,
    T_nb_point=2*T_nb_point,
    space_interval=space_interval,
    X_nb_point=2*X_nb_point,
    c=c,
    psi=psi_0,
    bctype="transparent",
);

## 3 - Schrödinger equation
We simulate the nonhomogeneous diffusion equation
\begin{equation}
    i\partial_t \psi + \Delta \psi + V\psi = 0,
\end{equation}
  
Here we simulate the following zero-potential equation:
$$
    \left\{\begin{array}{l}\partial_t \psi^{\text {int }}-i\partial_{xx} \psi^{\text {int }}=0, \quad(x, t) \in \Omega_{\text{PeriodToEmulate}}, \\
     \partial_{\mathbf{n}} \psi^{\text {int }}+e^{-i \pi / 4} \partial_t^{1 / 2} \psi^{\text {int }}=0, \quad(x, t) \in \Sigma_{\text{PeriodToEmulate}}, \\
      \psi^{\text {int }}(x, 0)=\psi_0(x), \quad x \in \Omega.\end{array}\right.
$$
I.e. for transparent boundary condition with $c = i, ~T=2, ~\delta_t= 10^{-3}, ~\Omega = [-10,10]$ and $\delta_x= 2\cdot10^-2$.

Here is the initialization with a single wave traveling towards the right side of the domain.

In [15]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from schroedinger_eq.schroedinger_simul_implementation import main_experience as schroedinger_simul_experience

# Time mesh
period_to_emulate = 2
T_nb_point  = 501

# Space mesh
space_interval  = (-10,10)
X_nb_point  = 501

# Initial condition (a default function is implemented)
def single_wave(x):
    psi_0_x = 2 / np.cosh(np.sqrt(2) * x) * np.exp(1j * 15 / 2 * x)  # See the article
    return psi_0_x

psi_0 = single_wave

#plot psi_0
interval_inf = space_interval[0]
interval_sup = space_interval[1]
x = np.linspace(interval_inf, interval_sup, X_nb_point)
plt.plot(x, np.real(psi_0(x)), label="real part")
plt.plot(x, np.imag(psi_0(x)), label="imaginary part")
plt.legend()
plt.title(r"Graph of $\psi_0$ with $||\psi_0||_{L_1}=$" + f'{np.round(np.sum(np.abs(psi_0(x)))/X_nb_point, 4)}')
plt.show()

### 3.2) Classic boundary conditions
#### 3.2.1) Dirichlet Boundary condition

In [16]:
bc_right_left_value = (0, 0)

schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="dirichlet",
    bc_right_left_value=bc_right_left_value,
);

#### 3.2.2) Neumann Boundary condition


In [17]:
bc_right_left_value = (0, 0)

# Launch code
schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="neumann",
    bc_right_left_value=bc_right_left_value,
);

### 3.3) Absorbing boundary condition

In [18]:
schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_basic",
);

### 3.4) Adding a potential

We add an attractive potential i.e. $V(x) = - 2\|x\|^2$ so that the border is not touched by the wave.

In [19]:
def space_potential(x, t):
    return -2.0 * np.abs(x) ** 2


V_independent = space_potential
bc_right_left_value = (0, 0)

# Launch code
schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_potential",
    bc_right_left_value=bc_right_left_value,
    V_independent=V_independent,
);

Trial with a lighter potential to let the wave touch the boundary : $V(x,t) = -0.1\|x\|^2$.

In [20]:
def space_potential(x, t):
    return -0.1 * np.abs(x) ** 2


V_independent = space_potential
bc_right_left_value = (0, 0)

# Launch code
schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_potential",
    bc_right_left_value=bc_right_left_value,
    V_independent=V_independent,
);

Now, we change the potential to be repulsive and compare the TBC that takes into account the potential `transparent_potential` and the one that does not `transparent_basic`.

In [21]:
def space_potential(x, t):
    return 0.1 * np.abs(x) ** 2


V_independent = space_potential
bc_right_left_value = (0, 0)

# Launch code
schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_basic",
    bc_right_left_value=bc_right_left_value,
    V_independent=V_independent,
)

schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_potential",
    bc_right_left_value=bc_right_left_value,
    V_independent=V_independent,
);

### 3.5) Perfectly Matched Layers


In [22]:
# Space mesh
space_interval = (-15,15)
X_nb_point = 501

# PML param (Included in space_interval)
delta = (5.0, 5.0)
sigma_0 = 0.05

schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="dirichlet",
    activate_PML=True,
    delta = delta,
    sigma_0 = sigma_0
);

### 3.6) Study of the error  
Let first verify the exact solution

In [23]:
from schroedinger_eq.schroedinger_exact_implementation import main_experience as schroedinger_exact_experience

# Convolution exact solution parameter
space_interval = (-10, 10)
convol_interval = (-20, 20)
convol_nb_point = 10000

res_exact = schroedinger_exact_experience(
        period_to_emulate=period_to_emulate,
        T_nb_point=T_nb_point,
        space_interval=space_interval,
        X_nb_point=X_nb_point,
        psi=psi_0,
        convol_interval=convol_interval,
        convol_nb_point=convol_nb_point,
        bool_plot=True,
    )

#### 3.6.1) ABC

*<u>Exact error:</u>*

In [24]:
from schroedinger_eq.schroedinger_study_error import study_of_error as schroedinger_error_study

# Time mesh
period_to_emulate = 2
T_nb_point  = 501

# Space mesh
space_interval  = (-10,10)
X_nb_point  = 501

psi_0 = single_wave

res_exact, res_simul, abs_error = schroedinger_error_study(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_basic",
    convol_interval=convol_interval,
)

In [25]:
%%capture
%matplotlib inline
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"

dt = period_to_emulate/T_nb_point
nb_frames = 30

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
def animate(i):
    ind = int(i * T_nb_point / nb_frames)
    # ind = i
    ax[0].clear()
    ax[1].clear()
    ax[0].plot(x, res_exact[ind], label = "exact")
    ax[0].plot(x, res_simul[ind], label = "simul")
    ax[1].plot(x, np.abs(np.abs(res_exact[ind]) - np.abs(res_simul[ind])), label = "error")
    ax[0].set_title('Plot of Schrodinger equation')
    ax[0].set_xlabel('x')
    ax[0].set_ylim(-2.1, 2.1)
    ax[0].set_ylabel('Time')
    ax[1].set_title('Error')
    ax[1].set_xlabel('x')
    # ax[1].set_ylim(-2.1, 2.1)
    ax[1].set_ylabel('Time')
    ax[0].legend()
    ax[1].legend()
    fig.suptitle(f"State at t={(ind*dt):.6g}", fontsize=16)

ani = animation.FuncAnimation(fig, animate, frames=nb_frames, repeat=False)
ani

*<u>Isolating reflections:</u>*  
As done for the wave equation, we compute an approximation on the normal domain with transparent boundary condition, stored in $\psi$, and compute an other approximation on a larger domain, named $\phi$. Then the reflections are isolated plotting the function $\psi - \phi|_\Omega$.

In [27]:
from schroedinger_eq.schroedinger_study_error import isolate_reflections

isolate_reflections(
    period_to_emulate=period_to_emulate,
    T_nb_point=501,
    space_interval=space_interval,
    X_nb_point=501,
    psi=psi_0,
    bctype="transparent_potential",
);

#### 3.6.2) PML

*<u>Exact error:</u>*  
Note that in case of PML, study is reduced in the area of interest (i.e. simulation space without damping layers).

In [28]:
delta = (0,3)
space_interval  = (-10, 13)

res_exact, res_simul, abs_error = schroedinger_error_study(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="dirichlet",
    activate_PML=True,
    delta=delta,
    convol_interval=convol_interval,
)

*<u>Isolating reflections:</u>*  

In [29]:
from schroedinger_eq.schroedinger_study_error import isolate_reflections

isolate_reflections(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="dirichlet",
    activate_PML=True,
    delta=delta,
);

#### 3.6.3) ABC + PML

*<u>Exact error:</u>*

In [30]:
delta = (0,3)
space_interval  = (-10, 13)

res_exact, res_simul, abs_error = schroedinger_error_study(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_basic",
    activate_PML=True,
    delta=delta,
    convol_interval=convol_interval,
)

*<u>Isolating reflections:</u>*  

In [31]:
from schroedinger_eq.schroedinger_study_error import isolate_reflections

isolate_reflections(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_basic",
    activate_PML=True,
    delta=delta,
);

#### Disgression: For glory, all together

In [32]:
def space_potential(x, t):
    return -0.1 * np.abs(x) ** 2

# Time mesh
period_to_emulate = 2
T_nb_point = 501

# Space mesh
space_interval = (-15,15)
X_nb_point = 1001

# PML param
delta = (5.0, 5.0)

V_independent = space_potential

from schroedinger_eq.schroedinger_simul_implementation import main_experience as schroedinger_simul_experience

schroedinger_simul_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    psi=psi_0,
    bctype="transparent_potential",
    activate_PML=True,
    delta = delta,
    V_independent=V_independent,
);

# Appendix: 
# E - Diffusion equation


### E.1 - Homogeneous diffusion equation
We simulate the wave equation
\begin{equation*}
    \partial_t \psi-c\partial_{xx} \psi=0,
\end{equation*}
with $c>0$.
We are going to make simulation with:
\begin{equation*}
T=0.1,~\#T_{\text{mesh}} = 500,~x\in[0,2],~\#X_{\text{mesh}} = 300 \text{ and }~c = 2.
\end{equation*}
  
Let's then define simulation variable.

In [33]:
import numpy as np

# Importation of the function
from homogoneous_diffusion_eq.diffusion_simul_implementation import (
    main_experience as diffusion_experience,
)

# Time mesh
period_to_emulate = 1
T_nb_point = 500

# Space mesh
space_interval = (0, 2)
X_nb_point = 300

# Celerity
c = 2

The proposed initial condition is a **double gaussian** as following.  

In [34]:
import matplotlib.pyplot as plt


# Initial condition (a default function is implemented)
def double_bump(x):
    psi_0_x = []
    a_1, b_1 = 0.1 * (space_interval[1] - space_interval[0]), 0.2 * (space_interval[1] - space_interval[0])
    a_2, b_2 = 0.6 * (space_interval[1] - space_interval[0]), 0.8 * (space_interval[1] - space_interval[0])
    for val in x:
        if a_1 <= val - space_interval[0] <= b_1:
            psi_0_x.append(np.abs(np.sin(val * np.pi / (b_1 - a_1))))
        elif a_2 <= val - space_interval[0] <= b_2:
            psi_0_x.append(np.abs(np.sin(val * np.pi / (b_2 - a_2))))
        else:
            psi_0_x.append(0)
    return np.array(psi_0_x)


psi_0 = double_bump

interval_inf = space_interval[0]
interval_sup = space_interval[1]
x = np.linspace(interval_inf, interval_sup, X_nb_point)
plt.plot(x, psi_0(x))
plt.title(
    r"Graph of $\psi_0$ with $||\psi_0||_{L_1}=$"
    + f"{np.round(np.sum(np.abs(psi_0(x)))/X_nb_point, 4)}"
)
plt.show()

#### E.1.1) Dirichlet boundary condition

In [35]:
# Value at boundaries
dirichlet_bcvalues = (0, 0)

# Launch code
diffusion_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    bctype="dirichlet",
    bcvalues=dirichlet_bcvalues,
);

#### E.1.2) Dirichlet boundary condition

In [36]:
# Value at boundaries
neumann_bcvalues = (0, 0)

# Launch code
diffusion_experience(
    period_to_emulate=period_to_emulate,
    T_nb_point=T_nb_point,
    space_interval=space_interval,
    X_nb_point=X_nb_point,
    c=c,
    psi=psi_0,
    bctype="neumann",
    bcvalues=neumann_bcvalues,
);

### E.2 - Nonhomogeneous diffusion equation
We simulate the nonhomogeneous diffusion equation
\begin{equation}
    \partial_t \psi-c\partial_{xx} \psi + V = 0,
\end{equation}
with $c>0$.
We are going to make simulation with:
\begin{equation*}
T=0.1,~\#T_{\text{mesh}} = 500,~x\in[0,2],~\#X_{\text{mesh}} = 300 \text{ and }~c = 2.
\end{equation*}
  
Let's then define simulation variable.

## F - Diffusion-Reaction equation
We simulate the diffusion-reaction diffusion equation
\begin{equation}
    \partial_t \psi-c\partial_{xx} \psi= cV\psi,
\end{equation}
with $c>0$.
We are going to make simulation with:
\begin{equation*}
T=0.1,~\#T_{\text{mesh}} = 500,~x\in[0,2],~\#X_{\text{mesh}} = 300 \text{ and }~c = 2.
\end{equation*}
  
Let's then define simulation variable.